In [1]:
import time
import datetime as dt
import pyvisa

In [2]:
# rm = visa.ResourceManager('C:\\Windows\\System32\\visa32.dll')

save_path = r"C:/Users/IPMU/Desktop/2025-gripper-run3/NFWF1948/"


# look up instruments
rm = pyvisa.ResourceManager()
print("resource manager : ", rm)
rl = rm.list_resources()
print("List of resources: \n", rl)

resource manager :  Resource Manager of Visa Library at C:\WINDOWS\system32\visa32.dll
List of resources: 
 ('ASRL1::INSTR', 'ASRL3::INSTR', 'GPIB1::12::INSTR', 'GPIB2::16::INSTR', 'GPIB3::2::INSTR')


In [3]:
for i,name in enumerate(rl):
    if "GPIB" in name:
        my_instrument = rm.open_resource(name)
        print("Resource[",i,"]:", name, "\n", my_instrument.query("*IDN?"))

Resource[ 2 ]: GPIB1::12::INSTR 
 LSCI,MODEL218S,LSB2EH,041813

Resource[ 3 ]: GPIB2::16::INSTR 
 KEITHLEY INSTRUMENTS INC.,MODEL 2700,4096641,B10  /A02  

Resource[ 4 ]: GPIB3::2::INSTR 
 NF Corporation,WF1948,9182457,Ver1.40



In [4]:
NFWF1948 = [rm.open_resource(rl[4])]

In [5]:

def Duration(freq):
    t = 29*400.*100. / (1.5 * freq)
    return t

def SignalON(name,CH):
    if CH == 1:
        name.write(':OUTPut1:STATe ON')
        print("turn on the CH1")
    elif CH == 2:
        name.write(':OUTPut2:STATe ON')
        print("turn on the CH2")
    else:
        print("Please select correct ch#")
        
def SignalOFF(name,CH):
    if CH == 1:
        name.write(':OUTPut1:STATe OFF')
        print("turn off the CH1")
    elif CH == 2:
        name.write(':OUTPut2:STATe OFF')
        print("turn off the CH2")
    else:
        print("Please select correct ch#")
        
def reset(name):
    name.write(':OUTPut1:STATe OFF')
    name.write(':OUTPut2:STATe OFF')
    print('reset all channel')
    
    
def pulse2rot(freq):
    print('WF freq. : %d Hz'%freq)
    print('Motor Driver freq. : %d Hz'%(freq/100.))
    print('Motor Rot. freq. : %f rot/s'%(freq/400./100.))
    print('Gripper moving speed : %f mm/s'%(freq/400./100.*1.5))
    return [freq,freq/100.,freq/400./100.,freq/400/100*1.5]

In [6]:
reset(NFWF1948[0])

reset all channel


In [7]:
#Initialize CH1
ind_freq = 0
freq = [500,1000,1500,2000,3000]
DURATIONTIME = Duration(freq[ind_freq])  # [sec] for frequency of daq
print(DURATIONTIME)

1546.6666666666667


In [8]:
NFWF1948[0].write(':SOURce1:VOLTage:LEVel:IMMediate:AMPLitude 5.0')
NFWF1948[0].write(':SOURce1:VOLTage:LEVel:IMMediate:OFFSet 2.5V')
NFWF1948[0].write(':SOURce1:FUNCtion:SHAPe SQUare')
NFWF1948[0].write(':SOURce1:FUNCtion:SQUare:DCYCle 50PCT')
NFWF1948[0].write(':SOURce1:FREQuency:FIXed '+str(freq[ind_freq])+'HZ')

32

In [9]:
#Initialize CH2
NFWF1948[0].write(':SOURce2:FUNCtion:SHAPe DC')
NFWF1948[0].write(':SOURce2:VOLTage:LEVel:IMMediate:OFFSet 5.0')

45

In [ ]:
# CCW CW
filename = save_path + "NF1948_" + dt.datetime.now().strftime("%Y%m%d%H%M%S") + "_log.txt"

arr_time = []

while True:
    
    arr_time = []
    
    SignalON(NFWF1948[0],2)
    SignalON(NFWF1948[0],1)
    arr_time.append(dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    
    time.sleep(DURATIONTIME)
    
    SignalOFF(NFWF1948[0],1)
    SignalOFF(NFWF1948[0],2)
    arr_time.append(dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    
    try:
        with open(filename, mode="a") as f:
            print("-------------------------------------------------",file=f)
            print(arr_time[3],"turn on", sep=", ", file=f)
            print(arr_time[4],"turn off", sep=", ", file=f)
            print(DURATIONTIME,"sec CCW rotation",sep=" ", file=f)
            rotlog = pulse2rot(freq[ind_freq])
            print('WF freq. : %d Hz'%rotlog[0], file=f)
            print('Motor Driver freq. : %d Hz'%rotlog[1], file=f)
            print('Motor Rot freq. : %f rot/s'%rotlog[2], file=f)
            print('Gripper moving speed : %f mm/s'%rotlog[3], file=f)
    except:
        pass
    
    reset(NFWF1948[0])
    arr_time = []
    reset(NFWF1948[0])
    SignalON(NFWF1948[0],1)
    arr_time.append(dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

    time.sleep(DURATIONTIME)

    SignalOFF(NFWF1948[0],1)
    arr_time.append(dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

    try:
        with open(filename, mode="a") as f:
            print("-------------------------------------------------",file=f)
            print(arr_time[0],"turn on", sep=", ", file=f)
            print(arr_time[1],"turn off", sep=", ", file=f)
            print(DURATIONTIME,"sec CW rotation",sep=" ", file=f)
            rotlog = pulse2rot(freq[ind_freq])
            print('WF freq. : %d Hz'%rotlog[0], file=f)
            print('Motor Driver freq. : %d Hz'%rotlog[1], file=f)
            print('Motor Rot freq. : %f rot/s'%rotlog[2], file=f)
            print('Gripper moving speed : %f mm/s'%rotlog[3], file=f)
    except:
        pass
    

turn on the CH2
turn on the CH1


In [ ]:
# CW CCW プラス方向が先
filename = save_path + "NF1948_" + dt.datetime.now().strftime("%Y%m%d%H%M%S") + "_log.txt"

arr_time = []

while True:
    arr_time = []
    reset(NFWF1948[0])
    SignalON(NFWF1948[0],1)
    arr_time.append(dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

    time.sleep(DURATIONTIME)

    SignalOFF(NFWF1948[0],1)
    arr_time.append(dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

    try:
        with open(filename, mode="a") as f:
            print("-------------------------------------------------",file=f)
            print(arr_time[0],"turn on", sep=", ", file=f)
            print(arr_time[1],"turn off", sep=", ", file=f)
            print(DURATIONTIME,"sec CW rotation",sep=" ", file=f)
            rotlog = pulse2rot(freq[ind_freq])
            print('WF freq. : %d Hz'%rotlog[0], file=f)
            print('Motor Driver freq. : %d Hz'%rotlog[1], file=f)
            print('Motor Rot freq. : %f rot/s'%rotlog[2], file=f)
            print('Gripper moving speed : %f mm/s'%rotlog[3], file=f)
    except:
        pass
    
    reset(NFWF1948[0])
    arr_time = []
    
    SignalON(NFWF1948[0],2)
    SignalON(NFWF1948[0],1)
    arr_time.append(dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    
    time.sleep(DURATIONTIME)
    
    SignalOFF(NFWF1948[0],1)
    SignalOFF(NFWF1948[0],2)
    arr_time.append(dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    
    try:
        with open(filename, mode="a") as f:
            print("-------------------------------------------------",file=f)
            print(arr_time[3],"turn on", sep=", ", file=f)
            print(arr_time[4],"turn off", sep=", ", file=f)
            print(DURATIONTIME,"sec CCW rotation",sep=" ", file=f)
            rotlog = pulse2rot(freq[ind_freq])
            print('WF freq. : %d Hz'%rotlog[0], file=f)
            print('Motor Driver freq. : %d Hz'%rotlog[1], file=f)
            print('Motor Rot freq. : %f rot/s'%rotlog[2], file=f)
            print('Gripper moving speed : %f mm/s'%rotlog[3], file=f)
    except:
        pass
    

reset all channel
turn on the CH1
turn off the CH1
WF freq. : 1000 Hz
Motor Driver freq. : 10 Hz
Motor Rot. freq. : 0.025000 rot/s
Gripper moving speed : 0.037500 mm/s
reset all channel
turn on the CH2
turn on the CH1
turn off the CH1
turn off the CH2
reset all channel
turn on the CH1
turn off the CH1
WF freq. : 1000 Hz
Motor Driver freq. : 10 Hz
Motor Rot. freq. : 0.025000 rot/s
Gripper moving speed : 0.037500 mm/s
reset all channel
turn on the CH2
turn on the CH1
turn off the CH1
turn off the CH2
reset all channel
turn on the CH1
turn off the CH1
WF freq. : 1000 Hz
Motor Driver freq. : 10 Hz
Motor Rot. freq. : 0.025000 rot/s
Gripper moving speed : 0.037500 mm/s
reset all channel
turn on the CH2
turn on the CH1
turn off the CH1
turn off the CH2
reset all channel
turn on the CH1
turn off the CH1
WF freq. : 1000 Hz
Motor Driver freq. : 10 Hz
Motor Rot. freq. : 0.025000 rot/s
Gripper moving speed : 0.037500 mm/s
reset all channel
turn on the CH2
turn on the CH1
turn off the CH1
turn off 

In [9]:
# CW rotation
filename = save_path + "NF1948_" + dt.datetime.now().strftime("%Y%m%d%H%M%S") + "_log.txt"
arr_time = []

reset(NFWF1948[0])

SignalON(NFWF1948[0],1)
arr_time.append(dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

time.sleep(DURATIONTIME)

SignalOFF(NFWF1948[0],1)
arr_time.append(dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

try:
    with open(filename, mode="a") as f:
        print("-------------------------------------------------")
        print(arr_time[0],"turn on", sep=", ", file=f)
        print(arr_time[1],"turn off", sep=", ", file=f)
        print(DURATIONTIME,"sec CW rotation",sep=" ", file=f)
        rotlog = pulse2rot(freq[ind_freq])
        print('WF freq. : %d Hz'%rotlog[0], file=f)
        print('Motor Driver freq. : %d Hz'%rotlog[1], file=f)
        print('Motor Rot freq. : %f rot/s'%rotlog[2], file=f)
        print('Gripper moving speed : %f mm/s'%rotlog[3], file=f)
except:
    pass



reset all channel
turn on the CH1
turn off the CH1
WF freq. : 1000 Hz
Motor Driver freq. : 10 Hz
Motor Rot. freq. : 0.025000 rot/s
Gripper moving speed : 0.037500 mm/s


In [10]:
# CCW rotation
filename = save_path + "NF1948_" + dt.datetime.now().strftime("%Y%m%d%H%M%S") + "_log.txt"
arr_time = []

reset(NFWF1948[0])

SignalON(NFWF1948[0],2)
SignalON(NFWF1948[0],1)
arr_time.append(dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

time.sleep(DURATIONTIME)

SignalOFF(NFWF1948[0],1)
SignalOFF(NFWF1948[0],2)
arr_time.append(dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

try:
    with open(filename, mode="a") as f:
        print("-------------------------------------------------")
        print(arr_time[0],"turn on", sep=", ", file=f)
        print(arr_time[1],"turn off", sep=", ", file=f)
        print(DURATIONTIME,"sec CCW rotation",sep=" ", file=f)
        rotlog = pulse2rot(freq[ind_freq])
        print('WF freq. : %d Hz'%rotlog[0], file=f)
        print('Motor Driver freq. : %d Hz'%rotlog[1], file=f)
        print('Motor Rot freq. : %f rot/s'%rotlog[2], file=f)
        print('Gripper moving speed : %f mm/s'%rotlog[3], file=f)
except:
    pass


reset all channel
turn on the CH2
turn on the CH1
